# import 

In [42]:
import pandas as pd 
import numpy as np

# importing credentials
import env

In [43]:
import os 

In [44]:
# importing credentials
import env

# import data

In [45]:
df = pd.read_csv('setsToSql',index_col= 0 )

In [46]:
df['set']

0              Base Set 
1              Base Set 
2              Base Set 
3              Base Set 
4              Base Set 
              ...       
3462    Supreme Victors 
3463    Supreme Victors 
3464    Supreme Victors 
3465    Supreme Victors 
3466    Supreme Victors 
Name: set, Length: 3467, dtype: object

In [47]:
stripper = lambda x: x.strip() if isinstance(x, str) else x

In [48]:
df['set'] = df['set'].apply(stripper)

# SQL CONNECTION AND MOVE

In [6]:
import mysql.connector
import os

# Retrieve the password from the environment variable
password = env.password
username = env.username

# Establish a connection
connection = mysql.connector.connect(
    host='localhost',
    user= username,
    password=password,
    database='PokemonTCG'
)

# Check if the connection is successful
if connection.is_connected():
    print("Connected to MySQL database")

# Perform database operations
# ...

# Close the connection when finished
connection.close()


Connected to MySQL database


# Serial Number to SQL

In [17]:
# Establish a connection
connection = mysql.connector.connect(
    host='localhost',
    user= username,
    password= password,
    database='PokemonTCG'
)

# Check if the connection is successful
if connection.is_connected():
    print("Connected to MySQL database")

# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Get the 'serial_number' values from the DataFrame
serial_numbers = df['serial_number'].tolist()

# Insert the 'serial_number' values into the 'cards' table
for serial_number in serial_numbers:
    query = "INSERT INTO cards (serial_number) VALUES (%s)"
    values = (serial_number,)
    #print(query)
    #print(values)
    cursor.execute(query, values)

# Commit the changes
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()


Connected to MySQL database


# Card Name to SQL

In [19]:
# Establish a connection
connection = mysql.connector.connect(
    host='localhost',
    user= username,
    password= password,
    database='PokemonTCG'
)

# Check if the connection is successful
if connection.is_connected():
    print("Connected to MySQL database")

# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Get the serial numbers and card names from the DataFrame
card_names = df['name'].tolist()

# Iterate over the serial numbers and card names to update the table
for serial_number, card_name in zip(serial_numbers, card_names):
    update_query = "UPDATE cards SET card_name = %s WHERE serial_number = %s"
    values = (card_name, serial_number)
    cursor.execute(update_query, values)

# Commit the changes
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()


Connected to MySQL database


# Rarity to SQL

In [20]:
# Establish a connection
connection = mysql.connector.connect(
    host='localhost',
    user= username,
    password= password,
    database='PokemonTCG'
)

# Check if the connection is successful
if connection.is_connected():
    print("Connected to MySQL database")

# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Iterate over the rows in the DataFrame
for index, row in df.iterrows():
    serial_number = row['serial_number']
    card_name = row['name']
    rarity = row['rarity']

    # Retrieve the rarity_id from the rarity table
    select_query = "SELECT rarity_id FROM rarity WHERE rarity_name = %s"
    cursor.execute(select_query, (rarity,))
    result = cursor.fetchone()

    if result is not None:
        rarity_id = result[0]

        # Update the cards table with the rarity_id
        update_query = "UPDATE cards SET rarity_id = %s WHERE serial_number = %s AND card_name = %s"
        update_values = (rarity_id, serial_number, card_name)
        cursor.execute(update_query, update_values)

# Commit the changes
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()


Connected to MySQL database


# Set to SQL

In [50]:
# Establish a connection
connection = mysql.connector.connect(
    host='localhost',
    user=username,
    password=password,
    database='PokemonTCG'
)

# Check if the connection is successful
if connection.is_connected():
    print("Connected to MySQL database")

# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Iterate over the rows in the DataFrame
for index, row in df.iterrows():
    serial_number = row['serial_number']
    card_name = row['name']
    rarity = row['rarity']
    set_name = row['set']

    # Retrieve the set_id from the sets table
    select_query = "SELECT set_id FROM sets WHERE set_name = %s"
    cursor.execute(select_query, (set_name,))
    result = cursor.fetchone()

    if result is not None:
        set_id = result[0]

        # Fetch all results from the first query to clear the unread result
        cursor.fetchall()

        # Retrieve the rarity_id from the rarity table
        select_rarity_query = "SELECT rarity_id FROM rarity WHERE rarity_name = %s"
        cursor.execute(select_rarity_query, (rarity,))
        rarity_result = cursor.fetchone()

        if rarity_result is not None:
            rarity_id = rarity_result[0]

            # Update the cards table with the set_id and rarity_id
            update_query = "UPDATE cards SET set_id = %s, rarity_id = %s WHERE serial_number = %s AND card_name = %s"
            update_values = (set_id, rarity_id, serial_number, card_name)
            cursor.execute(update_query, update_values)

# Commit the changes
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()


Connected to MySQL database


In [38]:
len(df['set'].unique().tolist())

31

,serial_number,name,type,rarity,set
0,1/102,Alakazam,Psychic,Rare Holo,Base Set
1,2/102,Blastoise,Water,Rare Holo,Base Set
2,3/102,Chansey,Colorless,Rare Holo,Base Set
3,4/102,Charizard,Fire,Rare Holo,Base Set
4,5/102,Clefairy,Colorless,Rare Holo,Base Set
...,...,...,...,...,...
3462,146/147,Rayquaza,Colorless,NaN,Supreme Victors
3463,147/147,Staraptor,Colorless,NaN,Supreme Victors
3464,148/147,Articuno,Water,NaN,Supreme Victors
3465,149/147,Moltres,Fire,NaN,Supreme Victors
